# Generate the Customer Profile (Enriched Zone: driver) using LLMs
- Create a Qualitative data analysis for Customer Habits
    - We have found some patterns that we want to dive deeper into:
      - trunk space
      - driving speed
      - hours worked
      - preferred pickup locations
      - average trip distance
      - crossing state lines
      - speaks spanish
      - vechicle cleanliness
      - vechile temperature
      - maximum passengers
      - conversation with customer
      - music playing
      - distracted driver
      - target pay

## Score all items in batches
- Find all records that have not been scored
- Score in a batch (we can do up to 10,000)
- The LLM temperature, max_output_tokens, top_p and top_k parameters have been set (locked for a deterministic value)
- Repeat until done

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
   SET extracted_customer_theme_json = NULL,
       extracted_customer_theme = NULL
 WHERE TRUE;
 */

In [ ]:
# need quotes around each category; otherwise, we get part of the category text
prompt = """
Classify the customer review as one of the following categories:
- "customer has small luggage"
- "customer has large luggage"
- "customer likes to drive fast"
- "customer likes to drive slow"
- "customer speaks spanish"
- "customer does not speak spanish"
- "customer likes a clean car"
- "customer likes the temperature warm"
- "customer likes the temperature cold"
- "customer likes conversation"
- "customer likes no conversation"
- "customer likes music"
- "customer likes quiet"

Review:
"""

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters so we are more deterministic and less creative/random responses
llm_temperature = 1
llm_max_output_tokens = 50
llm_top_p = 0
llm_top_k = 1

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
  SET extracted_customer_theme_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT trip_id,
                      customer_id,
                      CONCAT(\"\"\"{prompt}\"\"\",customer_review_text) AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
                WHERE (extracted_customer_theme_json IS NULL
                       OR
                       JSON_VALUE(extracted_customer_theme_json, '$.predictions[0].content') IS NULL
                       )
                  AND customer_review_text IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE customer_review.trip_id = child.trip_id
  AND customer_review.customer_id = child.customer_id;
  """.format(batch_size = batch_size,
             prompt = prompt,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
         WHERE (extracted_customer_theme_json IS NULL
                OR
                JSON_VALUE(extracted_customer_theme_json, '$.predictions[0].content') IS NULL
                )
           AND customer_review_text IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)

  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
   SET extracted_customer_theme = JSON_VALUE(extracted_customer_theme_json, '$.predictions[0].content')
 WHERE extracted_customer_theme_json IS NOT NULL
   AND extracted_customer_theme IS NULL;

In [ ]:
%%bigquery

SELECT trip_id, customer_review_text, extracted_customer_theme
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
 WHERE extracted_customer_theme_json IS NOT NULL
   AND extracted_customer_theme IS NOT NULL
LIMIT 20;

In [ ]:
%%bigquery

SELECT extracted_customer_theme, count(*) AS cnt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
 WHERE extracted_customer_theme_json IS NOT NULL
   AND extracted_customer_theme IS NOT NULL
GROUP BY 1;
